# BioConceptVec Tutorial

This tutorial provides a fundemental introduction to our BioConceptVec models. It illustrates (1) how to load the model, (2) how to get concept vectors, and (3) how to search top K similar concepts. [Source](https://github.com/ncbi/BioConceptVec/blob/master/bioconcept_tutorial.ipynb)

## 1. Prerequisites

Install gensim to load BioConceptVec.

In [1]:
from gensim.models import KeyedVectors
import os, sys, json, numpy as np

## 2. Load BioConceptVec

Go to data.ipynb to download the data. Then set the path to the data directory.

In [2]:
YOUR_BIOCONCEPTVEC_PATH = '../embeddings/bioconceptvec_glove.bin'
YOUR_JSON_PATH = '../embeddings/concept_glove.json'

Let's create a function using gensim to load BioConceptVec.

In [3]:
def load_embedding(path, binary):
    embedding = KeyedVectors.load_word2vec_format(path, binary=binary)
    print('embedding loaded from', path)
    return embedding

Let's load any one version of BioConceptVec. This might take a few minutes to load.

In [4]:
model = load_embedding(YOUR_BIOCONCEPTVEC_PATH, binary=True)

embedding loaded from ../embeddings/bioconceptvec_glove.bin


## 3. Load concept vectors only (an alternative approach)

If you only need concept vectors rather than other common word vectors, you could also load the json file that contains concept vectors only.

In [4]:
with open(YOUR_JSON_PATH) as json_file:  
    concept_vectors = json.load(json_file)
print('load', len(concept_vectors), 'concepts')

load 402712 concepts


In [ ]:
import pandas as pd
keys = concept_vectors.keys()
# Turn keys into a dataframe
df = pd.DataFrame(list(keys)[:500], columns=['concept'])

## 4. Get concept vectors

Now you could specify a concept ID and get the vector.

For the complete BioConceptVec model, you could use:

In [6]:
concept_vec = model['Gene_2997']
concept_vec

array([-0.205002, -0.858642,  0.293454,  0.420697,  0.819198, -0.014607,
        0.648112,  0.375288,  0.402923, -0.486146,  0.061995, -0.597637,
       -0.18606 , -0.030133,  0.057708,  0.0928  , -0.231601, -0.102536,
        0.113617,  0.096256, -0.716546,  0.129387,  0.097503,  0.516359,
        0.150461, -0.128564,  0.617396, -0.70626 , -0.214449, -0.609342,
       -0.610461,  0.202288,  0.509137,  0.601489,  0.49775 ,  0.046024,
        0.477214,  0.170123,  0.8417  ,  0.35232 ,  0.084167,  0.006656,
        0.052083,  0.27484 , -0.222268,  0.738473, -0.021782,  0.917502,
        0.208649,  0.21989 , -0.262444,  0.110046,  0.838382,  0.243271,
        0.584187, -0.17198 , -0.35021 ,  0.255658,  0.495955, -0.438681,
       -0.480301, -0.998345, -0.303257,  0.60441 , -0.235623,  0.19677 ,
       -0.311972,  0.406385,  0.067994,  0.487211, -0.016605, -0.42778 ,
       -1.023635,  0.342992, -0.103875,  0.076305,  0.890983,  0.101807,
       -0.019141, -0.469659, -0.081776, -0.074963, 

For the json file, it is a dictionary, so the code is exactly the same:

In [7]:
np.array(concept_vectors['Gene_2997'])

array([-0.20500199, -0.85864198,  0.29345399,  0.420697  ,  0.81919801,
       -0.014607  ,  0.648112  ,  0.37528801,  0.40292299, -0.486146  ,
        0.061995  , -0.597637  , -0.18606   , -0.030133  ,  0.057708  ,
        0.0928    , -0.231601  , -0.102536  ,  0.113617  ,  0.096256  ,
       -0.716546  ,  0.12938701,  0.097503  ,  0.51635897,  0.150461  ,
       -0.128564  ,  0.617396  , -0.70626003, -0.214449  , -0.60934198,
       -0.610461  ,  0.202288  ,  0.50913697,  0.60148901,  0.49775001,
        0.046024  ,  0.47721401,  0.170123  ,  0.84170002,  0.35231999,
        0.084167  ,  0.006656  ,  0.052083  ,  0.27484   , -0.222268  ,
        0.738473  , -0.021782  ,  0.91750199,  0.20864899,  0.21989   ,
       -0.26244399,  0.110046  ,  0.83838201,  0.24327099,  0.58418697,
       -0.17197999, -0.35021001,  0.255658  ,  0.49595499, -0.43868101,
       -0.48030099, -0.99834502, -0.30325699,  0.60440999, -0.235623  ,
        0.19677   , -0.31197199,  0.406385  ,  0.067994  ,  0.48

Similarly, you could use the concept IDs that we provided in the json file to get more concept vectors.

## 5.Compute the similarity between concepts

Now we can use the concept vectors to find similar concepts.

First let's create a function to calculate the Cosine similarity

In [8]:
def cosine(a, b):
    norm1 = np.linalg.norm(a)
    norm2 = np.linalg.norm(b)
    return np.dot(a, b) / (norm1 * norm2)

For example, for the interleukin 10 gene (Gene_3586), let's find out which gene is more similar to it, interleukin 4 gene (Gene_3565) or HUWE1 (Gene_10075)?

In [9]:
cosine(model['Gene_3586'], model['Gene_3565'])

0.88254434

In [10]:
cosine(model['Gene_3586'], model['Gene_10075'])

0.1340888

The results show that interleukin 4 gene is more similar to interleukin 10 gene. Indeed, they share GO terms.

## 6. Find top K similar terms via BioConceptVec

You can also use the embedding to find out the top K similar terms:

In [9]:
model.most_similar(positive=['Gene_432248'], topn=3)

[('Gene_399342', 0.6710575222969055),
 ('Gene_399275', 0.6615089774131775),
 ('Gene_378586', 0.6413488984107971)]

## Read in equations

### Read in mappings

In [1]:
import pickle   
# Load from pickle file
with open('/Users/danielgeorge/Documents/work/ml/bioconceptvec-explorer/bioconceptvec-explorer/mappings/concept_descriptions.pkl', 'rb') as f:
    concept_descriptions = pickle.load(f)
concept_descriptions

{'Disease_MESH_D001845': 'Bone Cysts',
 'Gene_2799940': 'matrix protein M2-1;matrix protein M2-2',
 'Gene_3726_54751': 'JunB proto-oncogene, AP-1 transcription factor subunit',
 'Disease_MESH_D000652': 'Amniotic Band Syndrome',
 'ProteinMutation_p_R450H_RS_189261858': ['uncharacterized LOC101928462',
  'thyroid stimulating hormone receptor'],
 'ProteinMutation_p_I123V': 'ProteinMutation I123V',
 'SNP_rs11944405': ['late endosomal/lysosomal adaptor, MAPK and MTOR activator 3'],
 'Chemical_MESH_C511970': 'M58373',
 'ProteinMutation_p_Q106E': 'ProteinMutation Q106E',
 'Gene_56941_116255': '5-hydroxymethylcytosine binding, ES cell specific',
 'SNP_rs12536544': ['Rac family small GTPase 1'],
 'Chemical_MESH_C558709': '16-(4-(3-(imidazol-1-yl)propoxy)-3-methoxybenzylidene)-5-androstene-3b,17b-diol',
 'Gene_100513670': 'non-SMC condensin I complex subunit G',
 'Gene_30244': 'engrailed homeobox 1a',
 'Chemical_MESH_C543977': 'wogonin-5-0-beta-D-glucuronide methyl ester',
 'Chemical_MESH_C02344

In [7]:
import pandas as pd

def process_line(line):
    parts = line.strip().split(' | ')

    expression = parts[0].split(': ')[1]
    solution = parts[1].split(': ')[1]
    similarity = float(parts[2].split(': ')[1])
    
    expression_parts = expression.split(' ')
    part1, operation1, part2, operation2, part3 = expression_parts
    
    return part1, operation1, part2, operation2, part3, solution, similarity

data = []

with open('../equations.txt', 'r') as file:
    for line in file:
        data.append(process_line(line))

df = pd.DataFrame(data, columns=['Part1', 'Operation1', 'Part2', 'Operation2', 'Part3', 'Solution', 'Similarity'])


In [8]:
df

,Part1,Operation1,Part2,Operation2,Part3,Solution,Similarity
0,Gene_6406_29106,-,SNP_rs34886328,+,Gene_100628162,Chemical_MESH_C104804,0.804830
1,SNP_rs12208888,-,ProteinMutation_p_F194Y,+,Gene_818533,Chemical_MESH_C582233,0.796501
2,ProteinMutation_p_G204S_RS_104894173,-,Gene_835_840_842,+,Chemical_MESH_C034706,ProteinMutation_p_H316V,0.808570
3,ProteinMutation_p_G44S,-,Chemical_MESH_C587588,+,Gene_107_108_109_196883_111_112_113_114_115,DNAMutation_g_12804T_C,0.759652
4,ProteinMutation_p_E53K_RS_863224013,-,ProteinMutation_p_C307S,+,Species_38736,ProteinMutation_p_C352Y_RS_193922566,0.763286
5,ProteinMutation_p_R199Q_RS_200850756,-,Chemical_MESH_C000334,+,Gene_492771,SNP_rs2284018,0.811054
6,ProteinMutation_p_H75W,-,SNP_rs139012426,+,ProteinMutation_p_E345R,ProteinMutation_p_T213M,0.751342
7,ProteinMutation_p_W13G_RS_139548132,-,Gene_110834_18518,+,Chemical_MESH_C473631,DNAMutation_c_544G_A,0.772074
8,DNAMutation_g_14206C_T,-,Chemical_MESH_C070377,+,Chemical_MESH_C083387,ProteinMutation_p_D708G,0.757347
9,Gene_66125,-,Gene_24974,+,Chemical_MESH_C115212,ProteinMutation_p_C228G,0.756925


In [10]:
# Go through Part1, Part2, Part3, Solution and map to concept descriptions. Search in concept_descriptions for the best match.
# Have both the original and the mapped version in the dataframe.

def get_best_match(concept_descriptions, concept):
    if concept in concept_descriptions:
        return concept_descriptions[concept]
    else:
        return concept

df['Part1_mapped'] = df['Part1'].apply(lambda x: get_best_match(concept_descriptions, x))
df['Part2_mapped'] = df['Part2'].apply(lambda x: get_best_match(concept_descriptions, x))
df['Part3_mapped'] = df['Part3'].apply(lambda x: get_best_match(concept_descriptions, x))
df['Solution_mapped'] = df['Solution'].apply(lambda x: get_best_match(concept_descriptions, x))
# Change the order of the columns to have the mapped ones next to the original ones
df = df[['Part1', 'Part1_mapped', 'Operation1', 'Part2', 'Part2_mapped', 'Operation2', 'Part3', 'Part3_mapped',  'Solution', 'Solution_mapped', 'Similarity']]
df

,Part1,Part1_mapped,Operation1,Part2,Part2_mapped,Operation2,Part3,Part3_mapped,Solution,Solution_mapped,Similarity
0,Gene_6406_29106,semenogelin 1,-,SNP_rs34886328,[MDM2 proto-oncogene],+,Gene_100628162,testis specific serine kinase 3,Chemical_MESH_C104804,None,0.804830
1,SNP_rs12208888,[],-,ProteinMutation_p_F194Y,ProteinMutation F194Y,+,Gene_818533,P-glycoprotein 6,Chemical_MESH_C582233,"2alpha,3beta,19alpha,23-tetrahydroxy-olean-12-...",0.796501
2,ProteinMutation_p_G204S_RS_104894173,[phytanoyl-CoA 2-hydroxylase],-,Gene_835_840_842,caspase 2,+,Chemical_MESH_C034706,glycyrrhetinyl-glycine conjugate,ProteinMutation_p_H316V,ProteinMutation H316V,0.808570
3,ProteinMutation_p_G44S,ProteinMutation G44S,-,Chemical_MESH_C587588,"5-(2-(2,4,6-trimethylphenyl)diazen-1-yl)quinol...",+,Gene_107_108_109_196883_111_112_113_114_115,adenylate cyclase 1,DNAMutation_g_12804T_C,DNAMutation g 12804T C,0.759652
4,ProteinMutation_p_E53K_RS_863224013,[fumarate hydratase],-,ProteinMutation_p_C307S,ProteinMutation C307S,+,Species_38736,Lithachne pauciflora,ProteinMutation_p_C352Y_RS_193922566,[low density lipoprotein receptor],0.763286
5,ProteinMutation_p_R199Q_RS_200850756,"[uncharacterized LOC124903011, D-amino acid ox...",-,Chemical_MESH_C000334,butanediol divinylether,+,Gene_492771,SNF8 subunit of ESCRT-II,SNP_rs2284018,[calcium voltage-gated channel auxiliary subun...,0.811054
6,ProteinMutation_p_H75W,ProteinMutation H75W,-,SNP_rs139012426,"[ATP8B1 antisense RNA 1, ATPase phospholipid t...",+,ProteinMutation_p_E345R,ProteinMutation E345R,ProteinMutation_p_T213M,ProteinMutation T213M,0.751342
7,ProteinMutation_p_W13G_RS_139548132,"[WARS2 antisense RNA 1, tryptophanyl tRNA synt...",-,Gene_110834_18518,"cholinergic receptor, nicotinic, alpha polypep...",+,Chemical_MESH_C473631,albuferon,DNAMutation_c_544G_A,DNAMutation c 544G A,0.772074
8,DNAMutation_g_14206C_T,DNAMutation g 14206C T,-,Chemical_MESH_C070377,CI 972,+,Chemical_MESH_C083387,ranatachykinin D,ProteinMutation_p_D708G,ProteinMutation D708G,0.757347
9,Gene_66125,"splicing factor 3b, subunit 5",-,Gene_24974,"RT1 class Ia, locus A2",+,Chemical_MESH_C115212,"2a alloantigen, human",ProteinMutation_p_C228G,ProteinMutation C228G,0.756925


In [6]:
# Turn df into an excel file
df.to_excel('equations.xlsx')